# HCS classification on preferred sensor/site combination

Imports

In [1]:
from IPython.display import Image
from IPython.display import display
import numpy as np

import ee
ee.Initialize()
import pandas as pd

import sys 
sys.path.append('/rheil/')
### Note: This path should lead to the root directory of the docker system running the 
### notebook. This is defined when you initialize the docker container. This enables you
### to import the custom scripts below.

import hcs_database as hcs_db
import cloud_mask

Next two cells set up the python and javascript code for the gee widget enabling slippy tile views. Eventually we should move this to a separate module that can be imported.

In [2]:
%%javascript
require.undef('geemap-widget');

define('geemap-widget', ["jupyter-js-widgets",'jquery', 'underscore'], function(widgets, $, _) {
    /**
     * A simple model to represent a layer on the map.
     *
     * @constructor
     */
    var Layer = Backbone.Model.extend({
        defaults: function() {
            return {
                config: {},
                type: undefined,
                visible: true
            };
        }
    });



    /**
     * A collection of layers.
     *
     * @constructor
     */
    var LayerCollection = Backbone.Collection.extend({
        model: Layer
    });



    /**
     * Override of the main widget model to intercept messages from Python
     * update Javascript state correctly.
     *
     * @constructor
     */
    var GoogleMapsModel = widgets.WidgetModel.extend({
        
        defaults: _.extend({}, widgets.DOMWidgetModel.prototype.defaults, {
            _model_name : 'GoogleMapsModel',
            _view_name : 'GoogleMapsView',
            _model_module : 'geemap-widget',
            _view_module : 'geemap-widget'
        }),
        
        /** @override */
        initialize: function() {
            this.listenTo(
                this, 'msg:custom', _.bind(this.handleMessage, this));
            // this.set('layers', new LayerCollection());
        },

        /**
         * Handle a message from Python.
         *
         * @param {!Object} payload Payload of the message.
         */
        handleMessage: function(payload) {
            if (!this.get('layers')) {
                this.set('layers', new LayerCollection());
            }
            switch(payload.action) {
                case 'addLayer':
                    this.get('layers').add({
                        config: payload.config,
                        type: payload.type
                    });
                    break;
                case 'removeLayer':
                    console.error('removeLayer not implemented');
                    break;
            }
        }
    });

    /**
     * A Google Maps API widget.
     *
     * @constructor
     */
    var GoogleMapsView = widgets.DOMWidgetView.extend({
        /**
         * Load the Maps API JS if needed, also prepare a deferred in case any
         * map methods are called before the map is ready.
         */
        initialize: function() {
            // Deferred to track for when the map is ready.
            this.mapsReadyDeferred = $.Deferred();

            // Dynamically adding Google Maps API JS here. Using a deferred to
            // track its load status as require returns as soon as the first
            // script loads and the Maps API triggers more scripts to append
            // which leaves a race condition where require thinks Maps API JS
            // is ready when it is not yet.
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            // Another instance of this view may have already loaded the Maps
            // API JS, do not try to load it twice.
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                };
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'],
                    function() {},
                    function() {});
            }
        },

        /**
         * Render the map.
         */
        render: function() {
            // We must wait until the Maps API JS is ready.
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                // Empty the views DOM. There seem to be some weird side
                // effects when you render more than one instance of this view
                // in the notebook. Cleaning the view DOM and deferring the map
                // initialization seems to work around this. It seesm almost if
                // map instances are sharing some DOM somehow.
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $([
                        '<div style="height: ',
                        this.model.get('height'),
                        '; width: ',
                        this.model.get('width'),
                        ';"></div>'
                    ].join(''));
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {
                            lat: this.model.get('lat'),
                            lng: this.model.get('lng')
                        },
                        zoom: this.model.get('zoom')
                    });
                    
                    this.initializeLayersControl();

                    // Notify the map when the map container changes size via
                    // the exposed properties in the model.
                    this.listenTo(
                        this.model, 'change:height', _.bind(function() {
                            this.$map.height(this.model.get('height'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));
                    this.listenTo(
                        this.model, 'change:width', _.bind(function() {
                            this.$map.width(this.model.get('width'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));

                    // Bind a change in the position of the map to the model.
                    google.maps.event.addListener(
                        this.map,
                        'bounds_changed',
                        _.bind(this.syncFromMap, this));

                    // Bind a change in the model (coming from the Python side)
                    // to the location of the map.
                    this.listenTo(
                        this.model,
                        'change:lat',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:lng',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:zoom',
                        _.bind(this.syncFromModel, this));

                    // Render the initial set of layers.
                    if (!this.model.get('layers')) {
                        this.model.set('layers', new LayerCollection());
                    }
                    this.model.get('layers').each(this.buildLayer, this);

                    // Bind to changes in the layers of the model to stay in
                    // sync.
                    this.listenTo(
                        this.model.get('layers'),
                        'add',
                        _.bind(this.buildLayer, this));
                    this.listenTo(
                        this.model.get('layers'),
                        'remove',
                        function() {
                            console.error('removeLayer not implemented');
                        });

                    // Even though a google.maps.Map instance should be ready
                    // immediately, it is not. This delay lets the stack clear
                    // and initial map bounds to be set.
                    _.delay(_.bind(function() {
                        this.mapsReadyDeferred.resolve();
                    }, this), 500);
                }, this));
            }, this));
        },

        //Based Tyler's earlier implementation
        LayersControl: function(widget, controlDiv, map) {
            // Set CSS styles for the DIV containing the control
            // Setting padding to 5 px will offset the control
            // from the edge of the map.
            controlDiv.style.padding = '5px';

            // Set CSS for the control border.
            this.$controlUI = $('<div />')
                .css('backgroundColor', 'white')
                .css('borderStyle', 'solid')
                .css('borderWidth', '1px')
                .css('cursor', 'pointer')
                .css('textAlign', 'center')
                .css('visibility', 'hidden')
                .appendTo($(controlDiv));
            
            // Set CSS for the control interior.
            var $controlContents = $('<div />')
                .css('fontFamily', 'Arial,sans-serif')
                .css('fontSize', '12px')
                .css('paddingLeft', '4px')
                .css('paddingRight', '4px')
                .css('paddingTop', '0px')
                .css('paddingBottom', '0px')
                .appendTo(this.$controlUI);
            
            this.$controlTable = $('<table />')
                .append($('<tr><td colspan=2>Layers</td></tr>'))
                .appendTo($controlContents);
        },
        
        initializeLayersControl: function() {
            // Create the DIV to hold the control and call the LayersControl() constructor
            // passing in this DIV.
    
            var layersControlDiv = document.createElement('div');
            
            this.layersControl = new this.LayersControl(this, layersControlDiv, this.map);
            
            layersControlDiv.index = 1;
            this.map.controls[google.maps.ControlPosition.TOP_RIGHT].push(layersControlDiv);
        },
        
        /**
         * Sync the values from the map into the model.
         */
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            // This is needed for the model to update the equivalent properties
            // on the Python instance of this view.
            this.model.save_changes();
        },

        /**
         * Move the map to match the values from the model.
         */
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },

        /**
         * Add a layer to the map based on its model.
         *
         * @param {!Layer} layer The layer to add.
         */
        buildLayer: function(layer) {
            switch(layer.get('type')) {
                case 'geojsondata':
                    this.addGeoJsonLayer(layer.get('config').data);
                    break;
                case 'geojsonurl':
                    this.loadGeoJsonLayer(layer.get('config').url);
                    break;
                case 'kmlurl':
                    this.loadKmlLayer(layer.get('config').url);
                    break;
                case 'earthengine':
                    this.addEarthEngineLayer(
                        layer.get('config').mapid, layer.get('config').token, layer.get('config').name, layer.get('config').visible);
                    break;
            }
        },

        /**
         * Adds GeoJSON to the map.
         *
         * @param {!Object} data A GeoJSON object.
         */
        addGeoJsonLayer: function(data) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.addGeoJson(data);
            }, this));
        },

        /**
         * Adds a URL location of GeoJSON to the map.
         *
         * @param {string} url The URL of the GeoJSON file to load.
         */
        loadGeoJsonLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.loadGeoJson(url);
            }, this));
        },

        /**
         * Add a KML layer to the map.
         *
         * @param {string} url The URL of the KML file to load.
         */
        loadKmlLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                new google.maps.KmlLayer({
                    url: url,
                    map: this.map
                });
            }, this));
        },

        /**
         * Add an Earth Engine layer to the map.
         *
         * @param {string} mapid The id of the Earth Engine layer.
         * @param {string} token The OAuth token to authenticate with.
         */
        addEarthEngineLayer: function(mapid, token, name, visible) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                var eeMapOptions = {
                    getTileUrl: function(tile, zoom) {
                        var url = [
                            'https://earthengine.googleapis.com/map',
                            mapid,
                            zoom,
                            tile.x,
                            tile.y
                        ].join('/');
                        url += '?token=' + token;
                        return url;
                    },
                    tileSize: new window.google.maps.Size(256, 256),
                    opacity: 1.0,
                };

                // Create the overlay map type.
                var mapType = new window.google.maps.ImageMapType(eeMapOptions);

                // Overlay the Earth Engine generated layer.
                this.map.overlayMapTypes.push(mapType);
                
                
                // Update layer visibility control
                var maxSlider = 100;

                function updateOpacity() {
                    mapType.setOpacity($checkbox[0].checked ? $slider[0].value / 100.0 : 0);
                }

                var $checkbox = $('<input type="checkbox">')
                    .prop('checked', visible)
                    .change(updateOpacity);

                var $slider = $('<input type="range" />')
                    .prop('min', 0)
                    .prop('max', maxSlider)
                    .prop('value', maxSlider)
                    .css('width', '60px')
                    .on('input', updateOpacity);

                // If user doesn't specify a layer name, create a default
                if (name === null) {
                    name = 'Layer ' + this.map.overlayMapTypes.length;
                }

                var $row = $('<tr />');
                $('<td align="left" />').append($checkbox).append(name).appendTo($row);
                $('<td />').append($slider).appendTo($row);
                
                this.layersControl.$controlUI.css('visibility', 'visible');
                this.layersControl.$controlTable.append($row);
            }, this));
        }
    });

    return {
        GoogleMapsModel: GoogleMapsModel,
        GoogleMapsView: GoogleMapsView
    }
});

<IPython.core.display.Javascript object>

In [3]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    """Google Maps API widget."""
    _model_name = traitlets.Unicode('GoogleMapsModel').tag(sync=True)
    _view_name = traitlets.Unicode('GoogleMapsView').tag(sync=True)
    _view_module = traitlets.Unicode('geemap-widget').tag(sync=True)
    _model_module = traitlets.Unicode('geemap-widget').tag(sync=True)
    lat = traitlets.CFloat(0).tag(sync=True)
    lng = traitlets.CFloat(0).tag(sync=True)
    zoom = traitlets.CInt(2).tag(sync=True)
    height = traitlets.CUnicode('300px').tag(sync=True)
    width = traitlets.CUnicode('400px').tag(sync=True)
    layers = traitlets.List([]).tag(sync=False)

    def addGeoJsonLayer(self, data):
        """Adds a dictionary of GeoJSON to the map.

        NOTE: It is likely if you are using a third party GeoJSON library you
        will have to first serialize the data into a simple dictionary before
        passing the data to this method.

        Args:
            data: A simple python dictionary of GeoJSON data.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsondata',
            'config': {'data': data}
        })

    def loadGeoJsonLayer(self, url):
        """Adds a URL location of GeoJSON to the map.

        Args:
            url: The URL of the GeoJSON file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsonurl',
            'config': {'url': url}
        })

    def loadKmlLayer(self, url):
        """Adds a KML layer to the map.

        Args:
            url: The URL of the KML file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'kmlurl',
            'config': {'url': url}
        })

    def addEarthEngineLayer(self, image, vis_params, name=None, visible=True):
        """Adds an Earth Engine layer to the map.

        Args:
            image: The ee.Image to display.
            vis_params: Dictionary of visualization parameters.
        """
        mapid = image.getMapId(vis_params)
        self.send({
            'action': 'addLayer',
            'type': 'earthengine',
            'config': {
                'mapid': mapid['mapid'],
                'token': mapid['token'],
                'name': name,
                'visible': visible
            }
        })

Load study area

In [11]:
key_csv = '/rheil/hcs/data/strata_key.csv'
key_df = pd.read_csv(key_csv)
from_vals = list(key_df['project_code'].astype(float).values)
to_vals = list(key_df['code_simpl'].astype(float).values)
site = 'app_jambi'
strata_img = ee.Image(hcs_db.app_rasters[site])
strata_img = strata_img.remap(from_vals, to_vals, 3)

In [9]:
site = 'app_all'
center_site = 'app_jambi'
coords = hcs_db.study_areas[center_site]['coordinates']
center_lng = np.mean([coords[0][0][0], coords[0][1][0]])
center_lat = np.mean([coords[0][0][1], coords[0][2][1]])
center = ee.Geometry.Point([center_lat, center_lng])

In [7]:
plot_coords = hcs_db.study_areas
plot_polygons = []
for site, coords in plot_coords.items():
    polygon = ee.Geometry.Polygon(coords['coordinates'])
    plot_polygons.append(polygon)
plot_fc = ee.FeatureCollection(plot_polygons)

Prep the study site. Can use any site that has a link to underlying fusion tables in hcs_database.py

In [7]:
# strata_map = ee.FeatureCollection(hcs_db.maps_dict[site], 'geometry')

Reduce to image

In [8]:
# strata_img = strata_map.reduceToImage(
#     properties = ['code_simpl'],
#     reducer = ee.Reducer.first())

# strata_img = strata_img.select(['first'], ['strata'])

Prep spectral data - Landsat 8

In [8]:
ic = ee.ImageCollection('LANDSAT/LC8_L1T_TOA_FMASK')
ic = ic.filterDate('2010-01-01', '2016-06-01')
ic = ic.filterMetadata(name = 'WRS_ROW', operator = 'less_than', value = 120)
ic = ic.filterBounds(plot_fc)
raw_img = ee.Image(ic.first())
ic_prepper = cloud_mask.prep_ic('l8', 'toa_fmask')
ic_masked = ic.map(ic_prepper)
clean_img = ic_masked.qualityMosaic('ndvi')

Example of how to use landsat 5 (could also use sentinel)

In [10]:
# ic = ee.ImageCollection('LANDSAT/LT5_L1T_TOA_FMASK')
# ic = ic.filterDate('2010-01-01', '2016-06-01')
# ic = ic.filterBounds(study_area)
# raw_img = ee.Image(ic.first())
# ic_prepper = cloud_mask.prep_ic('l5', 'toa_fmask')
# ic_masked = ic.map(ic_prepper)
# clean_img = ic_masked.qualityMosaic('ndvi')

Sample data

In [14]:
training = clean_img.addBands(strata_img).sample(numPixels = 500, seed = 0, scale = 30, 
                                                 region = plot_fc)
classifier = ee.Classifier.cart().train(training, 'strata')
training.getInfo()

{'columns': {},
 'features': [{'geometry': None,
   'id': '50',
   'properties': {'aerosol': 0.09935031831264496,
    'blue': 0.0761149600148201,
    'cirrus': 0.0011877448996528983,
    'green': 0.056962497532367706,
    'ndvi': 0.7553800344467163,
    'nir': 0.23918311297893524,
    'pan': 0.04894518479704857,
    'red': 0.03333122283220291,
    'remapped': 1,
    'swir1': 0.0841817632317543,
    'swir2': 0.030262870714068413,
    'tir1': 296.052978515625,
    'tir2': 293.452392578125},
   'type': 'Feature'},
  {'geometry': None,
   'id': '77',
   'properties': {'aerosol': 0.11013893783092499,
    'blue': 0.08693969994783401,
    'cirrus': 0.0008728379034437239,
    'green': 0.08032447099685669,
    'ndvi': 0.8380120396614075,
    'nir': 0.4782487750053406,
    'pan': 0.06679541617631912,
    'red': 0.042149100452661514,
    'remapped': 0,
    'swir1': 0.15226507186889648,
    'swir2': 0.04763881862163544,
    'tir1': 295.2870178222656,
    'tir2': 291.33056640625},
   'type': 'Featu

Accuracy of classifier on training data

In [13]:
classified = clean_img.classify(classifier)
trainAccuracy = classifier.confusionMatrix()
export = ee.batch.Export.table.toDrive(ee.FeatureCollection(ee.Feature(coords, {'matrix': trainAccuracy})))
export.start()
# print(trainAccuracy.accuracy().getInfo())
# trainAccuracy.getInfo()

EEException: Invalid GeoJSON geometry.

Out of sample accuracy

In [13]:
validation = clean_img.addBands(strata_img).sample(numPixels = 1000, seed = 1, scale = 30, region = study_area)
validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('strata', 'classification')
print(testAccuracy.accuracy().getInfo())
testAccuracy.getInfo()

NameError: name 'study_area' is not defined

Display spectral data, official stratification, and gee classification

In [ ]:
# Create map widgets.
map_a = GoogleMapsView(lng = center_lng, lat = center_lat, zoom = 11, height = '250px', width = '800px')

# Add the layers to the maps.
map_a.addEarthEngineLayer(image=clean_img.select(['nir', 'swir1', 'red']), vis_params={'min': 0, 'max': .6}, 
                         name = 'Landsat', visible = True)
map_a.addEarthEngineLayer(image=strata_img, vis_params={'min': 0, 'max': 10}, 
                          name='Stratification', visible=True)
map_a.addEarthEngineLayer(image=classified, vis_params={'min': 0, 'max': 10}, 
                          name='Classified', visible=True)

# Display map
display(map_a)


In [ ]:
# Create map widgets.
map_a = GoogleMapsView(lng = center_lng, lat = center_lat, zoom = 11, height = '250px', width = '800px')
map_b = GoogleMapsView(height = map_a.height, width = map_a.width)
map_c = GoogleMapsView(height = map_a.height, width = map_a.width)

# Link the map widgets.
widgets.jslink((map_a, 'lat'), (map_b, 'lat'))
widgets.jslink((map_a, 'lng'), (map_b, 'lng'))
widgets.jslink((map_a, 'zoom'), (map_b, 'zoom'))

widgets.jslink((map_a, 'lat'), (map_c, 'lat'))
widgets.jslink((map_a, 'lng'), (map_c, 'lng'))
widgets.jslink((map_a, 'zoom'), (map_c, 'zoom'))

# Add the layers to the maps
map_a.addEarthEngineLayer(image=clean_img.select(['nir', 'swir1', 'red']), vis_params={'min': 0, 'max': .6})
map_b.addEarthEngineLayer(image=strata_img, vis_params={'min': 0, 'max': 10})
map_c.addEarthEngineLayer(image=classified, vis_params={'min': 0, 'max': 10})

# Display
display(map_a)
display(map_b)
display(map_c)